In [1]:
import os;
import sys;
import random;

import numpy as np;
import cv2;
import matplotlib.pyplot as plt;

import tensorflow as tf
from tensorflow import keras;

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [3]:
image_size=(512,512);

def down_block(x,filters,kernel_size=(3,3),padding="same",strides=1):
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(x);
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(c);
    p = keras.layers.MaxPool2D((2,2),(2,2))(c);
    
    return c,p

def up_block(x,skip,filters,kernel_size=(3,3),padding="same",strides=1):
    us=keras.layers.UpSampling2D((2,2))(x);
    concat = keras.layers.Concatenate()([us,skip])
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(concat);
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(c);
    return c;

def bottleneck(x,filters,kernel_size=(3,3),padding="same",strides=1):
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(x);
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(c);
    return c;

In [4]:
def UNet():
    f = [16,32,64,128,256,512];
    
    inputs = keras.layers.Input(image_size+(3,))
    
    p0=inputs;
    c1,p1 = down_block(p0,f[0])  
    c2,p2 = down_block(p1,f[1]) 
    c3,p3 = down_block(p2,f[2])
    c4,p4 = down_block(p3,f[3])
    c5,p5 = down_block(p4,f[4])
    
    
    bn = bottleneck(p5,f[5])
    
    u5= up_block(bn,c5,f[4])
    u4= up_block(u5,c4,f[3])  
    u3= up_block(u4,c3,f[2])
    u2= up_block(u3,c2,f[1])
    u1= up_block(u2,c1,f[0])
    
    outputs = keras.layers.Conv2D(1,(1,1),padding="same",activation='sigmoid')(u1);
    model = keras.models.Model(inputs,outputs)
    
    return model

In [5]:
model = UNet();
model.compile(optimizer = "adam",loss="binary_crossentropy",metrics=["acc",f1_m]);
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 16) 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [6]:
model.save("unet.hdf5")